In [240]:
import requests
import json
import pandas as pd
from datetime import datetime
import statistics
from IPython.display import HTML, display
from wimbledon.vis.Visualise import DataHandlers

In [241]:
with open('github.token', 'r') as f:
    token = f.read().strip()

headers = {"Authorization": "Bearer " + token}


def run_query(query):  # A simple function to use requests.post to make the API call. Note the json= section.
    request = requests.post('https://api.github.com/graphql', json={'query': query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("Query failed to run by returning code of {}. {}".format(request.status_code, query))


GitHub emojis
=======

Project issues that members of the REG team have selected, "LAUGH" are the ones they would most like to be assigned to, "THUMBS_UP" they would also be happy with, "THUMBS_DOWN" they would not be happy with.

In [242]:
emojis = {'CONFUSED': '😕',
          'EYES': '👀',
          'HEART': '❤️',
          'HOORAY': '🎉',    
          'ROCKET': '🚀',
          'THUMBS_DOWN': '❌',
          'THUMBS_UP': '👍',
          'LAUGH': '✅'}

In [243]:
fc = DataHandlers.Forecast()  # get data from forecast

Enpoint: whoami | Time: 0.4795894699636847 
Enpoint: people/475627 | Time: 0.4724083309993148 

AUTHENTICATED USER:
Ed Chalstrey echalstrey@turing.ac.uk

CLIENTS
Enpoint: clients | Time: 0.49121608497807756 
PROJECTS
Enpoint: projects | Time: 0.7506305130082183 
ROLES
Enpoint: roles | Time: 1.32158765202621 
PEOPLE
Enpoint: people | Time: 0.5975445740041323 
PLACEHOLDERS
Enpoint: placeholders | Time: 0.4714971919893287 
MILESTONES
Enpoint: milestones | Time: 0.4527273399871774 
ASSIGNMENTS
Enpoint: assignments | Time: 0.8213435010402463 
DONE! (5.9s)


REG team
=====

We need a list of the team members full names from Forecast to compare with their names on GitHub. They match most of the time except when there is no GitHub name. Mapping below covers the rest of the team. We need to update this list if someone new is added to Forecast who doesn't have a matching GitHub name.

Find a team member's availability for a date range
---

In [244]:
def get_person_availability(fc, person, start_date, end_date):
    """Get the mean of a person's FTE proportion available for the start to end datetime objects"""
    peopledf = 1 - fc.people_totals.resample('MS').mean()  # pandas df for team members availability
    if isinstance(person, str):
        try:
            person = fc.get_person_id(person)
        except:
            return 0.0
    peopledf = peopledf[(peopledf.index >= start_date) & (peopledf.index <= end_date)]
    try:
        availability_range = peopledf[person]
    except:
        return 0.0
    average_availability = statistics.mean(availability_range)
    return round(average_availability, 2)

In [245]:
start_date = datetime(2019, 10, 1)
end_date = datetime(2020, 10, 1)
print(get_person_availability(fc, "Ed Chalstrey", start_date, end_date))
print(get_person_availability(fc, 458589, start_date, end_date))

0.46
1.0


Projects should have a consistent resource required number over the months they aren't 0, but may be useful to look at the resource requirement over a specific date range:

In [246]:
def get_project_requirement(fc, project, start_date, end_date):
    """Get the mean of a project's FTE requirement for the start to end datetime objects"""
    projectdf = fc.project_resourcereq.resample('MS').mean()  # pandas df for team members availability
    if isinstance(project, str):
        try:
            project = fc.get_project_id(project)
        except:
            return 0.0
    projectdf = projectdf[(projectdf.index >= start_date) & (projectdf.index <= end_date)]
    try:
        requirement_range = projectdf[project]
    except:
        return 0.0
    average_requirement = statistics.mean(requirement_range)
    return round(average_requirement, 2)

In [247]:
print(get_project_requirement(fc, "CHANCE", start_date, end_date))
print(get_project_requirement(fc, 2061048, start_date, end_date))

0
0.23


Create queries with GraphQL to get the emojis for each GitHub issue
===

https://developer.github.com/v4/guides/forming-calls/

In [248]:
query = """
{
  repository(owner:"alan-turing-institute", name:"Hut23") {
    issue(number:X) {
          number
          title
          url

          reactionGroups {
            content
            users(first:20) {
                edges {
                    node {
                        login
                        name
                    }
                }
            }
            }
    }
  }
}
"""

Check if a GitHub issue has emojis, based on the result of GraphQL query
----

In [249]:
def query_result_contains_emojis(query_result):
    for reaction in query_result['data']['repository']['issue']['reactionGroups']:
        if len(reaction['users']['edges']) > 0:
            return True
    return False

In [250]:
def get_preference_data(fc):
    """Get each team members preference emoji for all projects with a GitHub issue"""
    issues = fc.projects["GitHub"].dropna()  # Get list of GitHub issues for projects
    gid_mapping = {  # People without their full names on github.
     'myyong': 'May Yong',
     'nbarlowATI': 'Nick Barlow',
     'thobson88': 'Timothy Hobson',
     'miguelmorin': 'Miguel Morin',
     'OscartGiles': 'Oscar Giles',
     'AshwiniKV': 'Ashwini Venkatasubramaniam',
    }
    emoji_mapping = {'CONFUSED': '😕',
          'EYES': '👀',
          'HEART': '❤️',
          'HOORAY': '🎉',    
          'ROCKET': '🚀',
          'THUMBS_DOWN': '❌',
          'THUMBS_UP': '👍',
          'LAUGH': '✅'}
    names = list(fc.people.full_name)
    preference_data = {
        "Person": names
    }
    for issue_num, project_id in zip(issues, issues.index):
        modified_query = query.replace("X", str(issue_num))
        result = run_query(modified_query)  # Execute the query
#         if query_result_contains_emojis(result)  # Could do this if we only want issues with emojis
        emojis = []
        for name in names: 
            emoji_name = None
            for reaction in result['data']['repository']['issue']['reactionGroups']:
                for edge in reaction['users']['edges']:
                    if edge['node']['name'] == name:
                        emoji_name = reaction['content']
                    if not emoji_name:
                        if edge['node']['login'] in gid_mapping:
                            if gid_mapping[edge['node']['login']] == name:
                                emoji_name = reaction['content']
            if emoji_name:
                emoji = emoji_mapping[emoji_name]
            else:
                emoji = "❓"
            emojis.append(emoji)
        preference_data[fc.get_project_name(project_id)] = emojis
    return preference_data

In [251]:
preference_data = get_preference_data(fc)

In [252]:
preference_data_df.loc[~(preference_data_df=="❓").all(axis=1)]
preference_data_df

Sargasso (Intel Sparse) DetectorChecker HSBC EDS  \
Person                                                                        
Oliver Strickson                                 ❓               ❓        ❓   
Evelina Gabasova                                 ❓               ❓        ❓   
James Geddes                                     ❓               ❓        ❓   
James Hetherington                               ❓               ❓        ❓   
Louise Bowler                                    ❓               ❓        ❓   
May Yong                                         ❓               ❓        ❓   
Nick Barlow                                      ❓               ❓        ❓   
Radka Jersakova                                  ❓               ❓        ❓   
Timothy Hobson                                   ❓               ❓        ❓   
Tomas Lazauskas                                  ❓               ❓        ❓   
Camila Rangel Smith                              ❓               ❓        ❓   
Sarah Gibson                                     ❓               ❓        ❓   
James Robinson                                   ❓               ❓        ❓   
Eric Daub                                        ❓               ❓        ❓   
Jim Madge                                        ❓               ❓        ❓   
Kasra Hosseini                                   ❓               ❓        ❓   
Oscar Giles                                      ❓               ❓        ❓   
Jack Roberts                                     ❓               ❓        ❓   
Ed Chalstrey                                     ❓               ❓        ❓   
Ashwini Venkatasubramaniam                       ❓               ❓        ❓   
Roly Perera                                      ❓               ❓        ❓   

                           AI for City Planning Reproducible Research PDQ  \
Person                                                                      
Oliver Strickson                              ❓                     ❓   ❓   
Evelina Gabasova                              ❓                     ❓   ❓   
James Geddes                                  ❓                     ❓   ❓   
James Hetherington                            ❓                     ❓   ❓   
Louise Bowler                                 ❓                     ❓   ❓   
May Yong                                      ❓                     ❓   ❓   
Nick Barlow                                   ❓                     ❓   ❓   
Radka Jersakova                               ❓                     ❓   ❓   
Timothy Hobson                                ❓                     ❓   ❓   
Tomas Lazauskas                               ❓                     ❓   ❓   
Camila Rangel Smith                           ❓                     ❓   ❓   
Sarah Gibson                                  ❓                     ❓   ❓   
James Robinson                                ❓                     ❓   ❓   
Eric Daub                                     ❓                     ❓   ❓   
Jim Madge                                     ❓                     ❓   ❓   
Kasra Hosseini                                ❓                     ❓   ❓   
Oscar Giles                                   ❓                     ❓   ❓   
Jack Roberts                                  ❓                     ❓   ❓   
Ed Chalstrey                                  ❓                     ❓   ❓   
Ashwini Venkatasubramaniam                    ❓                     ❓   ❓   
Roly Perera                                   ❓                     ❓   ❓   

                           Crypto Federated Learning (SHEEP) The Turing Way  \
Person                                                                        
Oliver Strickson                                           ❓              ❌   
Evelina Gabasova                                           ❓              ✅   
James Geddes                                               ❓              ❓   
James Hetherington                        

In [253]:
preference_data_df["CHANCE"]["Ed Chalstrey"]

'✅'

Get the projects for which resources (team members) are required and display availability and preference for each team member who has with reacted with an emoji on the GitHub issue
----

In [254]:
def get_preferences(fc, preference_data_df, first_date=False, last_date=False, person=False, project=False, positive_only=False, emojis_only=False):
    resreqdf = fc.project_resourcereq.resample('MS').mean() # grouped by month and mean taken
    if person:
        names = [person]
    else:
        names = list(preference_data_df.index)
    data = {
        "Person": names
    }
    issues = fc.projects["GitHub"].dropna().values
    if project:
        if isinstance(project, str):
            try:
                project = fc.get_project_id(project)
            except:
                pass
    for project_id in resreqdf:  # some of these have no GitHub issue
        if not project or project == project_id:
            date_indices = resreqdf.index[resreqdf[project_id] > 0]
            if len(date_indices) > 0:  # if at least one month in the dataframe has a resource requirement of more than 0 FTE
                issue_num = fc.projects.loc[project_id, "GitHub"]
                if issue_num in issues:  # if this project has a GitHub issue
                    first_resreq_date = date_indices[0].strftime("%Y-%m-%d")
                    last_resreq_date = date_indices[-1].strftime("%Y-%m-%d")
                    resreq = get_project_requirement(fc, project_id, first_resreq_date, last_resreq_date)
                    project_title = fc.projects.loc[project_id, "name"]
                    emoji_data = []
                    for name in names:
                        person_availability = get_person_availability(fc, name, first_resreq_date, last_resreq_date)
                        percentage_availability = round((person_availability / resreq) * 100, 2)
                        emoji = preference_data_df[project_title][name]
                        if (not person and not project) or not positive_only or emoji == '✅' or emoji == '👍':
                            if emojis_only:
                                emoji_data.append(emoji)
                            else:
                                emoji_data.append(emoji + " " + str(percentage_availability) + "% (" + str(person_availability) + " / " + str(round(resreq, 2)) + ")")
#                         if project and positive_only and (emoji == '❌' or emoji == '❓'):
#                             print(name, emoji)
#                             data["Person"].remove(name)
#                         else:
#                             print(name, emoji)
                            
                    if (not person and not project) or not positive_only or len(emoji_data) > 0:
                        data[project_title] = emoji_data
    preferences = pd.DataFrame(data).set_index('Person').sort_index().sort_index(axis=1)
    return preferences

All projects with team members preferences and their average availability between the start of project resource requirement and project end date
----

In [255]:
preferences = get_preferences(fc, preference_data_df, emojis_only=True)

In [256]:
preferences["Census 2021 backbrief"]["Ed Chalstrey"]

'👍'

In [257]:
# preferences["CHANCE"]["Ed Chalstrey"]

In [258]:
css = """<style>
.tableFixHead {
          overflow: scroll;
          max-height: 100%;
          max-width: 100%;
        }
thead th {
          position: sticky;
          top: 0;
        }
tbody th {
          position: sticky;
          left: 0;
        }
thead th:first-child {
          left: 0;
          z-index: 1;
}
</style>"""
emoji_table = preferences.to_html()
html_table = css + """<div class="tableFixHead">""" + emoji_table + """</div>"""
# print(html_table)
with open("preferences.html", "w") as preferences_file:
    preferences_file.write(html_table)
display(HTML(html_table))

In [259]:
preferences_with_availability = get_preferences(fc, preference_data_df)

In [260]:
emoji_table2 = preferences_with_availability.to_html()
display(HTML(emoji_table2))

In [261]:
get_preferences(fc, preference_data_df, person="Ed Chalstrey", positive_only=True)

All in One Cancer imaging optimisation Census 2021 backbrief  \
Person                                                                      
Ed Chalstrey                   ✅ 25.0% (0.25 / 1.0)   👍 0.0% (0.0 / 0.28)   

             Interpretable AI for health Machine learning for cancer  \
Person                                                                 
Ed Chalstrey        👍 100.0% (1.0 / 1.0)        ✅ 43.0% (0.43 / 1.0)   

             Micro-location analytics SAIL multimorbidity  \
Person                                                      
Ed Chalstrey       ✅ 0.0% (0.0 / 1.0)  👍 0.0% (0.0 / 0.5)   

             Scaleable Monte Carlo Solar Panel Detection     Urban Agriculture  
Person                                                                          
Ed Chalstrey    👍 0.0% (0.0 / 1.0)  ✅ 66.0% (0.33 / 0.5)  👍 34.0% (0.17 / 0.5)

In [262]:
get_preferences(fc, preference_data_df, project="All in One Cancer imaging optimisation")

All in One Cancer imaging optimisation
Person                                                           
Ashwini Venkatasubramaniam                   👍 23.0% (0.23 / 1.0)
Camila Rangel Smith                          👍 100.0% (1.0 / 1.0)
Ed Chalstrey                                 ✅ 25.0% (0.25 / 1.0)
Eric Daub                                    ❌ 83.0% (0.83 / 1.0)
Evelina Gabasova                              👍 50.0% (0.5 / 1.0)
Jack Roberts                                 👍 67.0% (0.67 / 1.0)
James Geddes                                 ❓ 47.0% (0.47 / 1.0)
James Hetherington                            ❓ 9.0% (0.09 / 1.0)
James Robinson                               ❌ 100.0% (1.0 / 1.0)
Jim Madge                                    ❌ 83.0% (0.83 / 1.0)
Kasra Hosseini                               👍 77.0% (0.77 / 1.0)
Louise Bowler                                👍 83.0% (0.83 / 1.0)
May Yong                                     ✅ 94.0% (0.94 / 1.0)
Nick Barlow                                  👍 94.0% (0.94 / 1.0)
Oliver Strickson                             👍 71.0% (0.71 / 1.0)
Oscar Giles                                  ✅ 100.0% (1.0 / 1.0)
Radka Jersakova                              👍 100.0% (1.0 / 1.0)
Roly Perera                                   ❌ 4.0% (0.04 / 1.0)
Sarah Gibson                                 👍 100.0% (1.0 / 1.0)
Timothy Hobson                                👍 50.0% (0.5 / 1.0)
Tomas Lazauskas                              👍 100.0% (1.0 / 1.0)

In [263]:
get_preferences(fc, preference_data_df, project="All in One Cancer imaging optimisation", positive_only=True)

ValueError: arrays must all be same length